## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-31-14-53-29 +0000,nyt,What’s Next for Andrew After He Loses ‘Prince’...,https://www.nytimes.com/2025/10/31/world/europ...
1,2025-10-31-14-39-44 +0000,startribune,Health worker layoffs avoided with promise of ...,https://www.startribune.com/health-worker-layo...
2,2025-10-31-14-39-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/trump-urge...
3,2025-10-31-14-38-06 +0000,wapo,California’s own Louvre heist: Jewelry and art...,https://www.washingtonpost.com/nation/2025/10/...
4,2025-10-31-14-33-51 +0000,nypost,Devastating historic tavern fire leads to majo...,https://nypost.com/2025/10/31/us-news/ohio-ove...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,58
521,new,18
400,china,14
161,trade,13
143,you,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
175,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...,137
351,2025-10-30-16-06-07 +0000,bbc,Trump hails 'amazing' meeting with China's Xi ...,https://www.bbc.com/news/articles/crl25xl1gjpo...,121
200,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...,120
357,2025-10-30-15-33-03 +0000,nyt,"Trump and Xi, Hoping to Ease Trade War, Agree ...",https://www.nytimes.com/2025/10/30/business/us...,114
2,2025-10-31-14-39-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/trump-urge...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
175,137,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...
289,59,2025-10-30-19-35-06 +0000,nypost,Disgraced Prince Andrew to be stripped of ‘pri...,https://nypost.com/2025/10/30/world-news/princ...
2,56,2025-10-31-14-39-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/trump-urge...
243,53,2025-10-30-21-34-58 +0000,nyt,Do You Rely on Food Stamps? Tell Us About How ...,https://www.nytimes.com/2025/10/30/us/snap-foo...
199,52,2025-10-30-23-42-12 +0000,nypost,Amazon shares soar as AI demand boosts cloud r...,https://nypost.com/2025/10/30/business/amazon-...
200,50,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...
324,47,2025-10-30-17-31-09 +0000,nypost,"Hurricane Melissa leaves trail of death, destr...",https://nypost.com/2025/10/30/world-news/hurri...
294,43,2025-10-30-19-17-05 +0000,nypost,Treasury Secretary Scott Bessent hails US-Chin...,https://nypost.com/2025/10/30/business/scott-b...
305,39,2025-10-30-18-40-15 +0000,nypost,Kamala Harris claims Biden ‘didn’t want that d...,https://nypost.com/2025/10/30/us-news/kamala-h...
44,36,2025-10-31-12-48-12 +0000,nypost,Eric Adams’ NYPD investment will bring number ...,https://nypost.com/2025/10/31/us-news/nyc-mayo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
